In [29]:
import os
import requests
from terra_notebook_utils import azure_auth

In [32]:
DOMAIN = 'dsde-prod.broadinstitute.org'
LEO_BASE_URL = f'https://leonardo.{DOMAIN}/api/apps/v2'
WORKSPACE_ID = os.environ['WORKSPACE_ID']

def get_token():
    """Get Azure access token"""
    return azure_auth.get_azure_access_token()

def headers():
    return {
        "Authorization": f"Bearer {get_token()}",
        "Accept": "application/json"
    }

def get_cbas_url():
    """Get url for CBAS"""
    uri = f"{LEO_BASE_URL}/{WORKSPACE_ID}?includeDeleted=false"
    response = requests.get(uri, headers=headers())
    status_code = response.status_code
    if status_code != 200:
        return response.text
    response_json = response.json()
    for item in response_json:
        if item['appType'] == 'WORKFLOWS_APP':
            return item['proxyUrls']['cbas']
    return json.loads(response.text)['proxyUrls']['wds']

def get_post_data(start, end):
    return {
  "run_set_name": f"chess-eval-games_{start}-{end}",
  "run_set_description": "",
  "method_version_id": "2df9dc6a-ddde-4643-bb9e-1eb2e1435897",
  "workflow_input_definitions": [
    {
      "input_name": "ChessEval.offset",
      "input_type": {
        "type": "primitive",
        "primitive_type": "Int"
      },
      "source": {
        "type": "record_lookup",
        "record_attribute": "offset"
      }
    },
    {
      "input_name": "ChessEval.pgn_file",
      "input_type": {
        "type": "primitive",
        "primitive_type": "File"
      },
      "source": {
        "type": "record_lookup",
        "record_attribute": "pgn_file"
      }
    }
  ],
  "workflow_output_definitions": [
    {
      "output_name": "ChessEval.black_accuracy",
      "output_type": {
        "type": "primitive",
        "primitive_type": "Float"
      },
      "destination": {
        "type": "record_update",
        "record_attribute": "black_accuracy"
      }
    },
    {
      "output_name": "ChessEval.white_accuracy",
      "output_type": {
        "type": "primitive",
        "primitive_type": "Float"
      },
      "destination": {
        "type": "record_update",
        "record_attribute": "white_accuracy"
      }
    }
  ],
  "wds_records": {
    "record_type": "games",
    "record_ids": list(map(str, range(start,end))),
  },
  "call_caching_enabled": False
}

def submit_to_cbas(cbas_url, body):
    uri = f"{cbas_url}/api/batch/v1/run_sets"
    response = requests.post(uri, json=body, headers=headers())
    status_code = response.status_code
    if status_code != 200:
        raise Exception(response.text)
    return response.json()
    

In [33]:
batch = 1000
start = 8000
end = 9000
# 17185 total

In [34]:
cbas_url = get_cbas_url()
i = start
while i < end:
    print(f"Submitting batch {i}..{i+batch-1}")
    data = get_post_data(i, i+batch)
    submit_to_cbas(cbas_url, data)
    i = i + batch
print("Done")
    

Submitting batch 8000..8999
Done
